# SPE10 Demo - Distributed version

In [1]:
addprocs(4);
using DistributedArrays
@everywhere using DistributedArrays
using ForwardDiff
@everywhere using ForwardDiff
using StaticArrays
@everywhere using StaticArrays
@everywhere include("Stencil.jl")
@everywhere include("stencilgmres.jl")
@everywhere include("reservoirfunc.jl")
@everywhere include("distributed.jl")
@everywhere include("res3d.jl")
@everywhere include("preconditioning.jl")
using BenchmarkTools

In [2]:
permdata = open("spe_perm.dat")
phidata  = open("spe_phi.dat")
raw1 = readdlm(phidata)
raw2 = readdlm(permdata)
close(permdata)
close(phidata)
porosity = reshape(raw1', 60, 220, 85)
lineperm = reshape(raw2', 3366000, 1)
kx = reshape(lineperm[1:1122000]', 60, 220, 85)
ky = reshape(lineperm[1122001:2244000]', 60, 220, 85)
kz = reshape(lineperm[2244001:end]', 60, 220, 85)
savepor = copy(porosity)
## Porosity pre-processing
for id in eachindex(porosity)
    if abs(porosity[id])<1e-3
        porosity[id] = 0.2
        kx[id], ky[id], kz[id] = 1e-10, 1e-10, 1e-10
    end
end
kraw = [SVector{3,Float64}([kx[i,j,k], ky[i,j,k], kz[i,j,k]]) for i in 1:60, j in 1:220, k in 1:85];

In [3]:
Nx = 30
Ny = 30
Nz = 30
offsetx = 1
offsety = 1
offsetz = 1;

In [4]:
@everywhere p_ref = 14.7 
@everywhere C_r   = 3e-6
@everywhere ϕ_ref = 0.2
#=everywhere=# ϕ     = porosity[offsetx:(Nx+offsetx-1), offsety:(Ny+offsety-1), offsetz:(Nz+offsetz-1)]
@everywhere S_wc, S_or = 0.2, 0.2
@everywhere k_r_w(x)   = ((x-S_wc)/(1-S_wc-S_or))^2
@everywhere k_r_o(x)   = (1 - (x-S_wc)/(1-S_wc-S_or))^2
@everywhere p_cow(x)   = 0
@everywhere C_water    = 3e-6
@everywhere C_oil      = 3e-6
@everywhere ρ_water(p) = 64.0*exp(C_water*(p-p_ref)) 
@everywhere ρ_oil(p)   = 53.0*exp(C_oil*(p-p_ref))   
@everywhere μ_water = 0.3 # cp
@everywhere μ_oil   = 3.0; # cp
saveper = (kx.+ky.+kz)./3
saveper = saveper[offsetx:(Nx+offsetx-1), offsety:(Ny+offsety-1), offsetz:(Nz+offsetz-1)];

In [5]:
## 3d model
Lx = 1200
Ly = 2200
Lz = 170
Δt = 1.0
Tf = 2000.0
Δx_d = distribute(fill(Lx/Nx, Nx, Ny, Nz))
Δy_d = distribute(fill(Ly/Ny, Nx, Ny, Nz), Δx_d)
Δz_d = distribute(fill(Lz/Nz, Nx, Ny, Nz), Δx_d)
z_d  = distribute(fill(12000.0, Nx, Ny, Nz), Δx_d)
ϕ_d = distribute(ϕ, Δx_d)
k_d = makegrid(distribute(kraw[offsetx:(Nx+offsetx-1), offsety:(Ny+offsety-1), offsetz:(Nz+offsetz-1)],Δx_d),7)
model = Reservoirmodel(Δt, Tf, (Δx_d, Δy_d, Δz_d), z_d, k_d, p_ref, C_r, ϕ_ref, ϕ_d, 
                k_r_w, k_r_o, p_cow, C_water, C_oil, ρ_water, ρ_oil, μ_water, μ_oil);

In [6]:
## Porosity proportional control
Total = 40.0*Nx
q = zeros(Nx, Ny, Nz, 2)
for i in (1,Nx), j in (1,Ny), k in 1:Nz
    q[i,j,k,2] = Total*(saveper[i,j,k]/sum(saveper[i,j,:]))/4
end
halfx, halfy = round(Int, Nx/2),round(Int, Ny/2)
for k in 1:Nz
    q[halfx,halfy,k,1] = -Total*(saveper[halfx,halfy,k]/sum(saveper[halfx,halfy,:]))
end 
q_d = distribute(q);

In [7]:
init_d      = distribute([SVector{2,Float64}([6000.0,0.2]) for i in 1:Nx, j in 1:Ny, k in 1:Nz],Δx_d)
testgrid_d   = makegrid(init_d, 7);
g_guess_d   = testgrid_d;

In [8]:
S_d   = getstencil(model, q_d, g_guess_d, testgrid_d)
res_d = getresidual(model, q_d, g_guess_d, testgrid_d);

# Parallel benchmark (residual, jacobian)

In [9]:
@btime getstencil(model, q_d, g_guess_d, testgrid_d);

  1.633 s (1194 allocations: 74.33 KiB)


In [10]:
@btime getresidual(model, q_d, g_guess_d, testgrid_d);

  390.331 ms (2348 allocations: 179.45 KiB)
